<a href="https://www.kaggle.com/code/fhloston/the-fremen?scriptVersionId=182361153" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
def convert_to_int(value):
    if pd.isnull(value):
        return 0
    value = str(value)
    if value.endswith('B'):
        return int(float(value[:-1]) * 1_000_000_000)
    elif value.endswith('M'):
        return int(float(value[:-1]) * 1_000_000)
    elif value.endswith('K'):
        return int(float(value[:-1]) * 1_000)
    return int(value)

In [2]:
def convert_to_float(value):
    if pd.isnull(value):
        return 0.0
    else:
        return round(float(value.replace('$', '').replace(',', '')), 2)

In [3]:
# Function to format numbers as millions or billions with two decimal places
def format_millions_billions(value):
    if pd.isnull(value):
        return '0'
    if value >= 1_000_000_000:
        return f"{value / 1_000_000_000:.2f}B"
    elif value >= 1_000_000:
        return f"{value / 1_000_000:.2f}M"
    else:
        return f"{value:.2f}"

In [4]:
import pandas as pd

import pandasql as psql

file_path = '/kaggle/input/ipo-stock/IPO.csv'

IPO = pd.read_csv(file_path)

IPO.head()

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,WAY,Waystar Holding Corp.,$21.50,$20.70,-3.72%,3.45B,967.50M,NASDAQ
1,7-Jun-24,RAPP,"Rapport Therapeutics, Inc.",$17.00,$20.80,22.35%,735.83M,136.00M,NASDAQ
2,7-Jun-24,CHEB,Chenghe Acquisition II Co.,$10.00,$10.02,0.20%,100.20M,75.00M,NYSEAMERICAN
3,6-Jun-24,GAUZ,Gauzy Ltd.,$17.00,$15.74,-7.41%,295.29M,75.00M,NASDAQ
4,6-Jun-24,FLYE,"Fly-E Group, Inc.",$4.00,$4.00,NaN,100.00M,9.00M,NASDAQ


In [5]:
# Specify the columns that need conversion
columns_to_convert = ['MarketCap', 'DealSize']
columns_to_float_convert = ['IPOPrice', 'Current']

# Convert specified columns to integer
for column in columns_to_convert:
    IPO[column] = IPO[column].apply(convert_to_int)

# Convert specified columns to float
for column in columns_to_float_convert:
    IPO[column] = IPO[column].apply(convert_to_float)

# Print the DataFrame to verify the conversion
IPO.head()

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,WAY,Waystar Holding Corp.,21.5,20.70,-3.72%,3450000000,967500000,NASDAQ
1,7-Jun-24,RAPP,"Rapport Therapeutics, Inc.",17.0,20.80,22.35%,735830000,136000000,NASDAQ
2,7-Jun-24,CHEB,Chenghe Acquisition II Co.,10.0,10.02,0.20%,100200000,75000000,NYSEAMERICAN
3,6-Jun-24,GAUZ,Gauzy Ltd.,17.0,15.74,-7.41%,295290000,75000000,NASDAQ
4,6-Jun-24,FLYE,"Fly-E Group, Inc.",4.0,4.00,NaN,100000000,9000000,NASDAQ


# Dates

In [6]:
start_date = '4-Jan-19'
end_date = '9-Jun-24'

# SPAC Companies

In [7]:
query = f"""
Select *
from IPO
where CompanyName like '%Acquisition%'
and IPODate between '{start_date}' AND '{end_date}'
"""

result = psql.sqldf(query, locals())

result['MarketCap'] = result['MarketCap'].apply(format_millions_billions)
result['DealSize'] = result['DealSize'].apply(format_millions_billions)

display(result)

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Jun-24,CHEB,Chenghe Acquisition II Co.,10.0,10.02,0.20%,100.20M,75.00M,NYSEAMERICAN
1,9-Feb-24,HLXB,Helix Acquisition Corp. II,10.0,10.34,3.40%,235.98M,160.00M,NASDAQ
2,6-Oct-23,SPKL,Spark I Acquisition Corp. Class A Ordinary Share,10.0,10.34,3.40%,169.80M,100.00M,NASDAQ
3,6-Oct-23,QETA,Quetta Acquisition Corporation,10.0,10.34,3.35%,92.47M,60.00M,NASDAQ
4,5-May-23,ALCY,Alchemy Investments Acquisition Corp 1,10.0,10.69,6.90%,160.03M,100.00M,NASDAQ
5,4-Oct-22,AQU,Aquaron Acquisition Corp.,10.0,10.95,9.50%,49.86M,50.00M,NASDAQ
6,7-Apr-22,DECA,Denali Capital Acquisition Corp.,10.0,7.00,-30.00%,49.77M,75.00M,NASDAQ
7,9-Feb-22,BYNO,byNordic Acquisition Corporation,10.0,11.27,12.70%,115.81M,150.00M,NASDAQ
8,9-Feb-22,CPAQ,Counter Press Acquisition Corporation,10.0,10.33,3.30%,95.54M,75.00M,NASDAQ
9,8-Feb-22,LIVB,LIV Capital Acquisition Corp. II,10.0,10.97,9.70%,127.36M,100.00M,NASDAQ


# REIT Companies

In [8]:
query = f"""
Select * 
from IPO 
where CompanyName like '%Trust%' 
and IPODate between '{start_date}' AND '{end_date}'
"""

result = psql.sqldf(query, locals())
display(result)

,IPODate,Symbol,CompanyName,IPOPrice,Current,Return,MarketCap,DealSize,Exchange
0,7-Oct-20,SQFT,"Presidio Property Trust, Inc.",5.0,0.82,-83.70%,11860000,2500000,NASDAQ


# MIN/MAX IPOPrice

In [9]:
query = f"""
Select Symbol, CompanyName, IPODate, max(IPOPrice), min(IPOPrice) 
from IPO
where IPODate between '{start_date}' AND '{end_date}'
group by Symbol, CompanyName, IPODate, Exchange
"""

result = psql.sqldf(query, locals())
display(result)

,Symbol,CompanyName,IPODate,max(IPOPrice),min(IPOPrice)
0,WAY,Waystar Holding Corp.,7-Jun-24,21.5,21.5
1,AFIB,"Acutus Medical, Inc.",6-Aug-20,18.0,18.0
2,AHR,"American Healthcare REIT, Inc.",7-Feb-24,12.0,12.0
3,AI,"C3.ai, Inc.",9-Dec-20,42.0,42.0
4,ALCY,Alchemy Investments Acquisition Corp 1,5-May-23,10.0,10.0
...,...,...,...,...,...
161,VIGL,"Vigil Neuroscience, Inc.",7-Jan-22,14.0,14.0
162,VMAC,"Vistas Media Acquisition Company, Inc.",7-Aug-20,10.0,10.0
163,VRM,"Vroom, Inc.",9-Jun-20,22.0,22.0
164,XHG,XChange Tec.Inc.,5-Nov-19,0.0,0.0


# MIN/MAX MarketCap

In [10]:
query = f"""
Select Symbol, CompanyName, IPODate, min(MarketCap), max(MarketCap) 
from IPO 
where IPODate between '{start_date}' AND '{end_date}'
group by Symbol, CompanyName, IPODate, Exchange
"""

result = psql.sqldf(query, locals())
display(result)

,Symbol,CompanyName,IPODate,min(MarketCap),max(MarketCap)
0,WAY,Waystar Holding Corp.,7-Jun-24,3450000000,3450000000
1,AFIB,"Acutus Medical, Inc.",6-Aug-20,1350000,1350000
2,AHR,"American Healthcare REIT, Inc.",7-Feb-24,763180000,763180000
3,AI,"C3.ai, Inc.",9-Dec-20,3560000000,3560000000
4,ALCY,Alchemy Investments Acquisition Corp 1,5-May-23,160030000,160030000
...,...,...,...,...,...
161,VIGL,"Vigil Neuroscience, Inc.",7-Jan-22,172890000,172890000
162,VMAC,"Vistas Media Acquisition Company, Inc.",7-Aug-20,0,0
163,VRM,"Vroom, Inc.",9-Jun-20,18850000,18850000
164,XHG,XChange Tec.Inc.,5-Nov-19,217980000,217980000


# Big Deals


In [11]:
query = f"""
Select Symbol, CompanyName, IPODate, DealSize
from IPO 
where IPODate between '{start_date}' and '{end_date}'
and DealSize > 100000000
order by DealSize desc
"""

result = psql.sqldf(query, locals())

result['DealSize'] = result['DealSize'].apply(format_millions_billions)

display(result)

,Symbol,CompanyName,IPODate,DealSize
0,KVUE,Kenvue Inc.,4-May-23,3.80B
1,DASH,"DoorDash, Inc.",9-Dec-20,3.37B
2,RKT,"Rocket Companies, Inc.",6-Aug-20,2.70B
3,WAY,Waystar Holding Corp.,7-Jun-24,967.50M
4,SCRM,Screaming Eagle Acquisition Corp.,6-Jan-22,750.00M
5,SCRM,Screaming Eagle Acquisition Corp.,6-Jan-22,750.00M
6,RXT,"Rackspace Technology, Inc.",5-Aug-20,703.50M
7,AHR,"American Healthcare REIT, Inc.",7-Feb-24,672.00M
8,AI,"C3.ai, Inc.",9-Dec-20,651.00M
9,NXT,Nextracker Inc.,9-Feb-23,638.40M


# MIN/MAX Big Deals

In [12]:
start_date = '6-Jun-00'
end_date = '8-Jun-24'

query = f"""
Select Symbol, CompanyName, IPODate, min(DealSize), max(DealSize) 
from IPO 
where IPODate between '{start_date}' and '{end_date}'
and DealSize >= 100000000
group by Symbol, CompanyName, IPODate, Exchange
"""

result = psql.sqldf(query, locals())

result['min(DealSize)'] = result['min(DealSize)'].apply(format_millions_billions)
result['max(DealSize)'] = result['max(DealSize)'].apply(format_millions_billions)

display(result)

,Symbol,CompanyName,IPODate,min(DealSize),max(DealSize)
0,WAY,Waystar Holding Corp.,7-Jun-24,967.50M,967.50M
1,AHR,"American Healthcare REIT, Inc.",7-Feb-24,672.00M,672.00M
2,AMLX,"Amylyx Pharmaceuticals, Inc.",7-Jan-22,190.00M,190.00M
3,ATAK,DIH Holding US Inc,7-Feb-22,200.00M,200.00M
4,BLCO,Bausch + Lomb Corporation,6-May-22,630.00M,630.00M
5,CINC,"CinCor Pharma, Inc.",7-Jan-22,193.60M,193.60M
6,CXAC,C5 Acquisition Corporation,7-Jan-22,250.00M,250.00M
7,FRLN,Freeline Therapeutics Holdings plc,7-Aug-20,132.35M,132.35M
8,KYTX,"Kyverna Therapeutics, Inc.",8-Feb-24,319.00M,319.00M
9,LIVB,LIV Capital Acquisition Corp. II,8-Feb-22,100.00M,100.00M


# Exchange Statistics

In [13]:
query = f"""
Select Exchange, count(Symbol)
from IPO 
where IPODate between '{start_date}' and '{end_date}'
"""

result = psql.sqldf(query, locals())
display(result)

,Exchange,count(Symbol)
0,NASDAQ,89


# Exchange with biggest IPO count

In [14]:
query = f"""
Select Exchange, count(Symbol)
from IPO 
where IPODate between '{start_date}' and '{end_date}'
group by Exchange
order by count(Symbol) desc
Limit 1
"""

result = psql.sqldf(query, locals())
display(result)

,Exchange,count(Symbol)
0,NASDAQ,45


# IPO Positive return

In [15]:
query = f"""
SELECT COUNT(Symbol) AS PositiveReturnCount 
FROM IPO 
WHERE Return > 0
"""

result = psql.sqldf(query, locals())
display(result)

,PositiveReturnCount
0,571


# Total market capitalization

In [16]:
start_date = '1-Jan-24'
end_date = '9-Jun-24'

query = f"""
SELECT SUM(MarketCap) AS TotalMarketCap 
FROM IPO
where IPODate between '{start_date}' and '{end_date}'
"""

result = psql.sqldf(query, locals())

result['TotalMarketCap'] = result['TotalMarketCap'].apply(format_millions_billions)

display(result)

,TotalMarketCap
0,1837.47B
